In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
import keras
print(keras.__version__)

2.0.8


In [3]:
image_simulator_dir='data/simulator/'
image_train_dir='data/simulator/train'
image_validation_dir='data/simulator/validation'

## red directory
image_red_light_dir='red/'
image_red_dir = image_simulator_dir + image_red_light_dir

## green directory
image_green_light_dir='green/'
image_green_dir = image_simulator_dir + image_green_light_dir

## yellow directory
image_yellow_light_dir='yellow/'
image_yellow_dir = image_simulator_dir + image_yellow_light_dir

## Generate more training dataset

In [4]:
datagen = ImageDataGenerator(
        rotation_range=0,
        width_shift_range=0.2,
        height_shift_range=0.2,
        #rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


def augmentated_image(_number_of_outputs, _image_path, _source_image, _image_train_dir, _save_prefix, _save_format):
    img = load_img(_image_path + _source_image)  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=_image_train_dir, 
                              save_prefix=_save_prefix, save_format=_save_format):
        i += 1
        if i > _number_of_outputs:
            break  # otherwise the generator would loop indefinitely

### Generate red light training dataset

In [5]:
augmentated_image(200, image_red_dir, 'red_0.jpg', image_train_dir + '/red', 'red', 'jpeg')
augmentated_image(200, image_red_dir, 'red_1.jpg', image_train_dir + '/red', 'red', 'jpeg')
augmentated_image(200, image_red_dir, 'red_2.jpg', image_train_dir + '/red', 'red', 'jpeg')
augmentated_image(200, image_red_dir, 'red_3.jpg', image_train_dir + '/red', 'red', 'jpeg')

### Generate green light training dataset 

In [6]:
augmentated_image(200, image_green_dir, 'green_0.jpg', image_train_dir + '/green', 'green', 'jpeg')
augmentated_image(200, image_green_dir, 'green_1.jpg', image_train_dir + '/green', 'green', 'jpeg')
augmentated_image(200, image_green_dir, 'green_2.jpg', image_train_dir + '/green', 'green', 'jpeg')
augmentated_image(200, image_green_dir, 'green_3.jpg', image_train_dir + '/green', 'green', 'jpeg')

### Generate yellow light training dataset

In [7]:
augmentated_image(200, image_yellow_dir, 'yellow_0.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(200, image_yellow_dir, 'yellow_1.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(200, image_yellow_dir, 'yellow_2.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(200, image_yellow_dir, 'yellow_3.jpg', image_train_dir + '/yellow', 'yellow', 'jpeg')

## Validation Data Set

### Generate red light validation dataset

In [8]:
augmentated_image(50, image_red_dir, 'red_0.jpg', image_validation_dir + '/red', 'red', 'jpeg')
augmentated_image(50, image_red_dir, 'red_1.jpg', image_validation_dir + '/red', 'red', 'jpeg')
augmentated_image(50, image_red_dir, 'red_2.jpg', image_validation_dir + '/red', 'red', 'jpeg')
augmentated_image(50, image_red_dir, 'red_3.jpg', image_validation_dir + '/red', 'red', 'jpeg')

### Generate green light validation dataset

In [9]:
augmentated_image(50, image_green_dir, 'green_0.jpg', image_validation_dir + '/green', 'green', 'jpeg')
augmentated_image(50, image_green_dir, 'green_1.jpg', image_validation_dir + '/green', 'green', 'jpeg')
augmentated_image(50, image_green_dir, 'green_2.jpg', image_validation_dir + '/green', 'green', 'jpeg')
augmentated_image(50, image_green_dir, 'green_3.jpg', image_validation_dir + '/green', 'green', 'jpeg')

### Generate yellow light validation dataset

In [10]:
augmentated_image(50, image_yellow_dir, 'yellow_0.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(50, image_yellow_dir, 'yellow_1.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(50, image_yellow_dir, 'yellow_2.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')
augmentated_image(50, image_yellow_dir, 'yellow_3.jpg', image_validation_dir + '/yellow', 'yellow', 'jpeg')

## Convolution Neural Network

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

# dimensions of our images.
img_width, img_height = 320, 320

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [17]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [18]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/simulator/train',  # this is the target directory
        target_size=(320, 320),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 17560 images belonging to 3 classes.


In [19]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/simulator/validation',
        target_size=(320, 320),
        batch_size=batch_size,
        class_mode='binary')

Found 6050 images belonging to 3 classes.


In [20]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('simulator_traffic_light_model.h5') # always save your weights after training or during training

Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[16,318,318,32]
	 [[Node: conv2d_1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_conv2d_1_input_0_2/_17, conv2d_1/kernel/read)]]
	 [[Node: metrics/acc/Mean_1/_55 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_899_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'conv2d_1/convolution', defined at:
  File "/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-p